In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.autograd import Variable
import tensorflow as tf
import torch
device = torch.device("cpu")

In [86]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch import optim
# 데이터 불러오기
df_all=pd.read_csv('../data_v3/감성점수와 일별뉴스 합친거.csv',encoding='utf-8')
df = df_all.iloc[:,[1,2,5]]

In [87]:
df

,경락단가,경락단가_평균,day_sentiment
0,3750.366093,3750.366093,0.000000
1,3547.574932,3547.574932,0.000000
2,3696.060897,3696.060897,-0.104167
3,3770.156364,3770.156364,-0.104167
4,3809.974684,3809.974684,-0.104167
...,...,...,...
620,4547.071429,4547.071429,0.175000
621,4373.212544,4373.212544,-0.116270
622,4278.649485,4278.649485,-0.116270
623,4322.080986,4322.080986,-0.116270


In [88]:
#df=df[['day_sentiment','경락단가_평균']]
df=df[['경락단가','경락단가','경락단가_평균']]

df.tail()


,경락단가,경락단가,경락단가_평균
620,4547.071429,4547.071429,4547.071429
621,4373.212544,4373.212544,4373.212544
622,4278.649485,4278.649485,4278.649485
623,4322.080986,4322.080986,4322.080986
624,4138.037190,4138.037190,4138.037190


In [132]:
# 7일간의 데이터가 입력으로 들어가고 batch size는 임의로 지정
seq_length = 5
batch = 100

# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
train_size = int(len(df)*0.7)
train_set = df[0:train_size]  
test_set = df[train_size-seq_length:]

# Input scale
scaler_x = MinMaxScaler()
scaler_x.fit(train_set.iloc[:, :-1])

train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])

# Output scale
scaler_y = MinMaxScaler()
scaler_y.fit(train_set.iloc[:, [-1]])

train_set.iloc[:, -1:] = scaler_y.transform(train_set.iloc[:, -1:])
test_set.iloc[:, -1:] = scaler_y.transform(test_set.iloc[:, -1:])


train_set = torch.FloatTensor(train_set.to_numpy())
test_set = torch.FloatTensor(test_set.to_numpy())

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [136]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

# 데이터셋 생성 함수
def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series)-seq_length):
        _x = time_series[i:i+seq_length, 0:1]
        if i == len(time_series)-seq_length-1:
            _y = [[time_series[len(time_series)-seq_length-1, 1:2][0]],[time_series[len(time_series)-seq_length-1, 1:2][0]],[time_series[len(time_series)-seq_length-1, 1:2][0]]]
            _y=np.array(_y)
        else:
            _y = time_series[i+seq_length:i+seq_length+3, 1:2]
        # print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

ValueError: only one element tensors can be converted to Python scalars

In [134]:
trainY

array([array([[0.2522236 ],
       [0.24482408],
       [0.2566514 ]], dtype=float32),
       array([[0.24482408],
       [0.2566514 ],
       [0.25857568]], dtype=float32),
       array([[0.2566514 ],
       [0.25857568],
       [0.2718726 ]], dtype=float32),
       array([[0.25857568],
       [0.2718726 ],
       [0.27665332]], dtype=float32),
       array([[0.2718726 ],
       [0.27665332],
       [0.24731068]], dtype=float32),
       array([[0.27665332],
       [0.24731068],
       [0.24248086]], dtype=float32),
       array([[0.24731068],
       [0.24248086],
       [0.2483569 ]], dtype=float32),
       array([[0.24248086],
       [0.2483569 ],
       [0.29187137]], dtype=float32),
       array([[0.2483569 ],
       [0.29187137],
       [0.30350387]], dtype=float32),
       array([[0.29187137],
       [0.30350387],
       [0.25248212]], dtype=float32),
       array([[0.30350387],
       [0.25248212],
       [0.27916574]], dtype=float32),
       array([[0.25248212],
       [0.27916

In [131]:
trainY

array([array([[0.2522236 ],
       [0.24482408],
       [0.2566514 ]], dtype=float32),
       array([[0.24482408],
       [0.2566514 ],
       [0.25857568]], dtype=float32),
       array([[0.2566514 ],
       [0.25857568],
       [0.2718726 ]], dtype=float32),
       array([[0.25857568],
       [0.2718726 ],
       [0.27665332]], dtype=float32),
       array([[0.2718726 ],
       [0.27665332],
       [0.24731068]], dtype=float32),
       array([[0.27665332],
       [0.24731068],
       [0.24248086]], dtype=float32),
       array([[0.24731068],
       [0.24248086],
       [0.2483569 ]], dtype=float32),
       array([[0.24248086],
       [0.2483569 ],
       [0.29187137]], dtype=float32),
       array([[0.2483569 ],
       [0.29187137],
       [0.30350387]], dtype=float32),
       array([[0.29187137],
       [0.30350387],
       [0.25248212]], dtype=float32),
       array([[0.30350387],
       [0.25248212],
       [0.27916574]], dtype=float32),
       array([[0.25248212],
       [0.27916

In [78]:
y

(-0.699831273296347, -0.699831273296347, -0.699831273296347)

In [121]:
trainX.shape

(432, 5, 1)

In [127]:
trainX_tensor

tensor([[[ 0.2591],
         [ 0.2073],
         [ 0.2452],
         [ 0.2642],
         [ 0.2744]],

        [[ 0.2073],
         [ 0.2452],
         [ 0.2642],
         [ 0.2744],
         [ 0.2522]],

        [[ 0.2452],
         [ 0.2642],
         [ 0.2744],
         [ 0.2522],
         [ 0.2448]],

        ...,

        [[ 0.2930],
         [ 0.2675],
         [ 0.2083],
         [-0.6999],
         [-0.6998]],

        [[ 0.2675],
         [ 0.2083],
         [-0.6999],
         [-0.6998],
         [-0.6998]],

        [[ 0.2083],
         [-0.6999],
         [-0.6998],
         [-0.6998],
         [-0.6998]]])

In [105]:
testY=np.concatenate(testY)

In [130]:
# 텐서로 변환
# trainX_tensor = torch.FloatTensor(trainX)
# trainY_tensor = torch.FloatTensor(trainY)
# testX_tensor = torch.FloatTensor(testX)
# testY_tensor = torch.FloatTensor(testY)

# 텐서 형태로 데이터 정의
dataset = TensorDataset(trainX_tensor, trainY_tensor)

# 데이터로더는 기본적으로 2개의 인자를 입력받으며 배치크기는 통상적으로 2의 배수를 사용
dataloader = DataLoader(dataset,
                        batch_size=batch,
                        shuffle=True,  
                        drop_last=True)

# 설정값
data_dim = 1
hidden_dim = 10 
output_dim = 1
learning_rate = 0.01
nb_epochs = 100
class LSTMModel(nn.Module):
    # # 기본변수, layer를 초기화해주는 생성자
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers):
        super(LSTMModel, self).__init__()
        self.seq_len = seq_len
        self.hidden_dim = hidden_dim
        self.layer_dim = layers

        # LSTM model 
        self.lstm = nn.LSTM(input_dim, hidden_dim, layers, batch_first=True) 

        self.fc = nn.Linear(hidden_dim, output_dim)
        
    # 학습 초기화를 위한 함수
    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layer_dim, self.seq_len, self.hidden_dim),
                torch.zeros(self.layer_dim, self.seq_len, self.hidden_dim))
    
    # 예측을 위한 함수
    def forward(self, x):
    
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(out[:, -1, :]) 
        return out

def train_model(model, train_df, num_epochs = None, lr = None, verbose = 10, patience = 10):
     
    criterion = nn.MSELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    nb_epochs = num_epochs
    
    # epoch마다 loss 저장
    train_hist = np.zeros(nb_epochs)

    for epoch in range(nb_epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        for batch_idx, samples in enumerate(train_df):

            x_train, y_train = samples
            x_train=x_train.to(device)
            y_train=y_train.to(device)
            # seq별 hidden state reset
            model.reset_hidden_state()
            
            # H(x) 계산
            outputs = model(x_train)
                
            # cost 계산
            loss = criterion(outputs, y_train)                    
            
            # cost로 H(x) 개선
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_cost += loss/total_batch
               
        train_hist[epoch] = avg_cost        
        
        if epoch % verbose == 0:
            print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost))
            
        # patience번째 마다 early stopping 여부 확인
        if (epoch % patience == 0) & (epoch != 0):
            
            # loss가 커졌다면 early stop
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping')
                
                break
            
    return model.eval(), train_hist

# 모델 학습
net = LSTMModel(data_dim, hidden_dim, seq_length, output_dim, 1).to(device)  
model, train_hist = train_model(net, dataloader, num_epochs = nb_epochs, lr = learning_rate, verbose = 20, patience = 10)

# epoch별 손실값
fig = plt.figure(figsize=(10, 4))
plt.plot(train_hist, label="Training loss")
plt.legend()
plt.show()

# 모델 저장    
PATH = "./Timeseries_LSTM_data-02-stock_daily_.pth"
torch.save(model.state_dict(), PATH)

# 불러오기
model = LSTMModel(data_dim, hidden_dim, seq_length, output_dim, 1).to(device)  
model.load_state_dict(torch.load(PATH), strict=False)
model.eval()

# 예측 테스트
with torch.no_grad(): 
    pred = []
    for pr in range(len(testX_tensor)):

        model.reset_hidden_state()

        predicted = model(torch.unsqueeze(testX_tensor[pr], 0).to(device) )
        predicted = torch.flatten(predicted)
        pred.append(predicted)
    # INVERSE
    pred_inverse = scaler_y.inverse_transform(np.array(pred).reshape(-1, 1))
    testY_inverse = scaler_y.inverse_transform(testY_tensor)

AssertionError: 

In [40]:
len(pred)

188

In [41]:
testY_tensor.shape

torch.Size([188, 1])

In [42]:
def MAE(true, pred):
    return np.mean(np.abs(true-pred))

print('MAE SCORE : ', MAE(pred_inverse, testY_inverse))

#MAE
from sklearn.metrics import mean_absolute_error

print("MAE",mean_absolute_error(testY_inverse,pred_inverse))

#MSE
from sklearn.metrics import mean_squared_error

print("MSE",mean_squared_error(testY_inverse,pred_inverse))

#RMSE
import numpy as np
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(testY_inverse,pred_inverse)
print("MSE",np.sqrt(MSE))
# sklearn 은 mse만 제공하기 때문에 rmse는 직접 만들어 써야한다.

#MAPE
import numpy as np

def MAPE(y_test, y_pred):
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
print("MAPE",MAPE(testY_inverse, pred_inverse))

NameError: name 'pred_inverse' is not defined